# Credibility/Sensitivity

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import statsmodels.api as sm
import os
import pathlib


from austen_plots.AustenPlot import AustenPlot

In [2]:
RANDOM_SEED = 42

def make_Q_model():
    return LinearRegression()
    return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=100, max_depth=5)

def make_g_model():
    return LogisticRegression(max_iter=1000)
    return RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=100, max_depth=5)

# helper functions to implement the cross fitting

def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    """
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    """
    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
        X_train = X.loc[train_index]
        A_train = A.loc[train_index]
        g = make_model()
        g.fit(X_train, A_train)

        # get predictions for split
        predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]

    assert np.isnan(predictions).sum() == 0
    return predictions


def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"

    """
    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
        X_train = X_w_treatment.loc[train_index]
        y_train = y.loc[train_index]
        q = make_model()
        q.fit(X_train, y_train)

        if output_type =='binary':
            predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
            predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
        elif output_type == 'continuous':
            predictions0[test_index] = q.predict(X0.loc[test_index])
            predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1


def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    # Double ML estimator for the ATT
    This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
    """

    if prob_t is None:
        prob_t = A.mean() # estimate marginal probability of treatment

    tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
    scores = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
    n = Y.shape[0] # number of observations
    std_hat = np.std(scores) / np.sqrt(n)

    return tau_hat, std_hat

### Single time period

In [3]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

# wf.drop('Unnamed: 0', axis = 1)
# city_yb.drop('Unnamed: 0', axis = 1)
# city_yb = city_yb.dropna()
# print(len(wf))
# wf = wf.merge(city_yb, on='city_code').dropna(
#     subset = ['sec_city', 'gdp_city', 'pgdp_city', 
#               'firm_city', 'gonglu', 'emit_ww', 'emit_so1', 'emi_dust1',
#               'aqi', 'pm']
# )
# print(len(wf))

wf["temp2"] = wf["temp"] ** 2
wf["l_aqi"] = np.log(1 + wf["aqi"])
wf["l_pm"] = np.log(1 + wf["pm"])
wf2020 = wf[(wf["daynum"] >= 8401) & (wf["daynum"]<= 8461)].dropna(
    subset = ['aqi', 'pm']
)
wf2020['cities'] = wf2020['city_code'].astype('category')
wf2020['days'] = wf2020['daynum'].astype('category')
wf2020 = pd.get_dummies(wf2020, drop_first=True)

fixed = ['treat']
city_fixed = []
time_fixed = []
for col in wf2020.columns:
    if 'cities' in col:
        city_fixed.append(col)
    if 'days' in col:
        time_fixed.append(col)
fixed = fixed + city_fixed + time_fixed
        
weather = ['prec', 'snow', 'temp', 'temp2']
city_economic = ['pop_city', 'sec_city', 'gdp_city' , 'pgdp_city', 'firm_city']
city_environmental = ['gonglu', 'emit_ww', 'emit_so1', 'emi_dust1']
out = ["aqi", "l_aqi", "pm", "l_pm"]

treated = wf2020[wf2020['treat'] == 1]
treated = treated[['daynum', 'city_code']].groupby('city_code')
first = treated.apply(lambda x: x.sort_values(by = 'daynum', ascending=True).head(1))

day, count = np.unique(first.daynum, return_counts = True)
treat_day = day[count == max(count)][0]

num_cities = {d:c for d,c in zip(day, count)}
first = {city:day for day, city in first.values}

wf2020 = wf2020.assign(first = [first.get(city, 0) for city in wf2020['city_code']])
group = wf2020[(wf2020['first'] == treat_day) | (wf2020['first'] == 0)]
wf2020['first'] = wf2020['first'].astype('category')
wf2020 = pd.get_dummies(wf2020, drop_first=True)

group['pre'] = group['daynum'] < treat_day
group = group.groupby(['city_code', 'pre']).mean().reset_index('pre')

compact = group[~group['pre']]
aqi = group['aqi'].values
compact['Y1-Y0'] = aqi[~group['pre']] - aqi[group['pre']]


compact = compact.reset_index()
outcome = compact['Y1-Y0']
treatment = compact['treat']
confounders = compact[city_fixed + time_fixed + weather]

g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=10)
Q0,Q1=outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, n_splits=10, output_type="continuous")

data_and_nuisance_estimates = pd.DataFrame({'g': g, 'Q0': Q0, 'Q1': Q1, 'A': treatment, 'Y': outcome})

tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

# for comparison, the point estimate without any covariate correction
outcome[treatment==1].mean()-outcome[treatment==0].mean()


/Users/jtf/anaconda3/envs/mamba-base-env/envs/nn_class/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The estimate is -19.861800850414966 pm 11.89715227874221


-18.886275852614084

#### Multiple time period

In [4]:
wf2020 = wf2020.assign(first = [first.get(city, 0) for city in wf2020['city_code']])
wf2020['A'] = (wf2020['daynum'] == wf2020['first']).astype('int64')

wf2020['diff'] = wf2020.sort_values(by = 'daynum')['aqi'] \
            - wf2020.sort_values(by = 'daynum').groupby('city_code')['aqi'].shift(1)
wf2020 = wf2020.dropna(subset= ['diff'])

outcome = wf2020['diff']
treatment = wf2020['A']
confounders = wf2020[['daynum'] + weather + city_fixed + time_fixed]

# specify a model for the conditional expected outcome

# TODO(victorveitch) the covariates have basically no predictive power, replace this example with something better

# make a function that returns a sklearn model for later use in k-folding
def make_Q_model():
    #return LinearRegression()
    return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators = 100, max_depth=10)
Q_model = make_Q_model()

# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_w_treatment = confounders.copy()
X_w_treatment["treatment"] = treatment

X_train, X_test, y_train, y_test = train_test_split(X_w_treatment, outcome, test_size=0.2)
Q_model.fit(X_train, y_train)
y_pred = Q_model.predict(X_test)


# specify a model for the propensity score

def make_g_model():
    #return LogisticRegression(max_iter=1000)
    return RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=100, max_depth=3)

g_model = make_g_model()
# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_train, X_test, a_train, a_test = train_test_split(confounders, treatment, test_size=0.2)
g_model.fit(X_train, a_train)
a_pred = g_model.predict_proba(X_test)[:,1]

# for comparison, the point estimate without any covariate correction
print(outcome[treatment==1].mean()-outcome[treatment==0].mean())

res = dict()
Q_model.fit(X_w_treatment, outcome)
g_model.fit(confounders, treatment)

for day in np.unique(wf2020['daynum']):
    df = wf2020[wf2020['daynum'] == day]
    outcome_t = df['diff']
    treatment_t = df['A']
    confounders_t = df[['daynum'] + weather + city_fixed + time_fixed]
    
    if df['A'].sum() == 0 or num_cities[day] < 2:
        continue
    
    X1 = confounders_t.copy()
    X0 = confounders_t.copy()
    X1["treatment"] = 1
    X0["treatment"] = 0
    
    Q0 = Q_model.predict(X0)
    Q1 = Q_model.predict(X1)
    g = g_model.predict_proba(confounders_t)[:,1]
    
    est, sd = att_aiptw(Q0, Q1, g, treatment_t, outcome_t)
    res[day] = (est, sd)
    
inv_var = np.array([1/v**2 for p,v in res.values()])
point = np.array([p for p,v in res.values()])    

tau_hat = (point * inv_var).sum()/inv_var.sum()
std_hat = np.sqrt(1/inv_var.sum())

print('%0.3f pm %0.3f' % (tau_hat, std_hat))

KeyboardInterrupt: 

## Parallel trends

Parametric multiple-time period

In [ ]:
# create week coefficient 
treated = wf2020[wf2020['treat'] == 1]
treated = treated[['daynum', 'city_code']].groupby('city_code')
first = treated.apply(lambda x: x.sort_values(by = 'daynum', ascending=True).head(1))
day, count = np.unique(first.daynum, return_counts = True)
treat_day = day[count == max(count)][0]
first = {city:day for day, city in first.values}
wf2020 = wf2020.assign(first = [first.get(city, 0) for city in wf2020['city_code']])
wf2020["week_coef"] = np.floor((wf2020["daynum"] - wf2020["first"])/7).astype(int)

# set -1 lead and untreated to NaN so they don't get week0 dummy
wf2020["week_coef"] = np.where((wf2020["week_coef"] == -1), np.NaN, wf2020["week_coef"])
wf2020["week_coef"][wf2020["first"] == 0] = np.NaN
wf2020["week_coef"] = wf2020["week_coef"].astype('category')
wf2020 = pd.get_dummies(wf2020)

week_coef = []
for col in wf2020.columns:
    if 'week_coef' in col:
        week_coef.append(col)


In [ ]:
for Yname in out:
    Y = wf2020[Yname]
    X = wf2020[fixed + weather + week_coef]
    fit = sm.OLS(Y, sm.add_constant(X)).fit()
    print(Yname)
    print(*list(zip([index for index in fit.params.index if 'week_coef' in index],
                    fit.params[[index for index in fit.params.index if 'week_coef' in index]], 
                   2*fit.bse[[index for index in fit.params.index if 'week_coef' in index]])), sep="\n")

check parallel trends on two-period subsets (23 and 16)

In [ ]:
# E[Y_{t+1} - Y_{t} | A=1, X] - E[Y_{t+1} - Y_{t} | A=0, X] = 0 in all pre-treatment periods t.
#  if you've fit a model for Q(a,x) = E[Y_{t+1} - Y_{t} | A=a, x] then you can plot diff(t) = 1/n \sum_i Q(1,x_i) - Q(0,x_i)



day_23 = day[count == max(count)][0]
wf23 = wf2020[(wf2020['first'] == day_23) | (wf2020['first'] == 0)]

# estimate for each day
day_diffs = []
day_list = np.sort(np.unique(wf23['daynum']))
for d in day_list:
    df = wf23[wf23['daynum'] == d]
    confounders_t = df[['daynum'] + weather + city_fixed + time_fixed]
    
    X1 = confounders_t.copy()
    X0 = confounders_t.copy()
    X1["treatment"] = 1
    X0["treatment"] = 0
    
    Q0 = Q_model.predict(X0)
    Q1 = Q_model.predict(X1)
    
    day_diffs.append(np.mean(Q1 - Q0))

list(zip(day_list, day_diffs))

In [ ]:
X1

In [ ]:
# estimate by week
weeks_list = []
week_diffs = []
for w in week_coef:
    wf23_week = wf23[wf23[w] == 1]
    if len(wf23_week) == 0:
        continue
    confounders_t = wf23_week[['daynum'] + weather + city_fixed + time_fixed]
    
    X1 = confounders_t.copy()
    X0 = confounders_t.copy()
    X1["treatment"] = 1
    X0["treatment"] = 0
    
    Q0 = Q_model.predict(X0)
    Q1 = Q_model.predict(X1)
    weeks_list.append(w)
    week_diffs.append(np.mean(Q1 - Q0))
    
list(zip(weeks_list, week_diffs))

In [ ]:
day_16 = day[3]

In [ ]:
day_16

### Model fit

In [ ]:
Q_model

## Covariate Influence Strength

#### Single time period

In [ ]:
def _convert_to_austen_format(nuisance_estimate_df: pd.DataFrame):
  austen_df = pd.DataFrame()
  austen_df['y']=nuisance_estimate_df['Y']
  austen_df['t']=nuisance_estimate_df['A']
  austen_df['g']=nuisance_estimate_df['g']
  A = nuisance_estimate_df['A']
  austen_df['Q']=A*nuisance_estimate_df['Q1'] + (1-A)*nuisance_estimate_df['Q0'] # use Q1 when A=1, and Q0 when A=0

  return austen_df

In [ ]:
covariate_groups = {
    'city_fixed': city_fixed,
    'time_fixed': time_fixed,
    'weather': weather,
    }

In [ ]:
# For each covariate group, refit the models without using that group
nuisance_estimates = {}
for group, covs in covariate_groups.items():
    remaining_confounders = confounders.drop(columns=covs)
    print(remaining_confounders)
    
    g = treatment_k_fold_fit_and_predict(make_g_model, X=remaining_confounders, A=treatment, n_splits=10)
    Q0, Q1 = outcome_k_fold_fit_and_predict(make_Q_model, X=remaining_confounders, y=outcome, A=treatment, n_splits=10, output_type="continuous")

    data_and_nuisance_estimates = pd.DataFrame({'g': g, 'Q0': Q0, 'Q1': Q1, 'A': treatment, 'Y': outcome})
    nuisance_estimates[group] = data_and_nuisance_estimates

In [ ]:
data_and_nuisance_path = 'data_and_nuisance_estimates.csv'
covariate_dir_path = 'covariates/'

In [ ]:
def _convert_to_austen_format(nuisance_estimate_df: pd.DataFrame):
  austen_df = pd.DataFrame()
  austen_df['y']=nuisance_estimate_df['Y']
  austen_df['t']=nuisance_estimate_df['A']
  austen_df['g']=nuisance_estimate_df['g']
  A = nuisance_estimate_df['A']
  austen_df['Q']=A*nuisance_estimate_df['Q1'] + (1-A)*nuisance_estimate_df['Q0'] # use Q1 when A=1, and Q0 when A=0

  return austen_df

In [ ]:
austen_data_and_nuisance = _convert_to_austen_format(data_and_nuisance_estimates)
austen_data_and_nuisance.to_csv(data_and_nuisance_path, index=False)

pathlib.Path(covariate_dir_path).mkdir(exist_ok=True)
for group, nuisance_estimate in nuisance_estimates.items():
  austen_nuisance_estimate = _convert_to_austen_format(nuisance_estimate)
  austen_nuisance_estimate.to_csv(os.path.join(covariate_dir_path,group+".csv"), index=False)

In [ ]:
target_bias = 8.00 # note: bias is specified as an absolute number
ap = AustenPlot(data_and_nuisance_path, covariate_dir_path)
p, plot_coords, variable_coords = ap.fit(bias=target_bias)
p

### Multiple time period

In [ ]:
nuisance_estimates

In [ ]:
nuisance_estimates

In [ ]:
# For each covariate group, refit the models without using that group
nuisance_estimates = {}
for group, covs in covariate_groups.items():
    # make model with covariates dropped
    Q_model = make_Q_model()
    g_model = make_g_model()
    
    remaining_confounders = confounders.drop(columns=covs)
    X_w_treatment_rem = remaining_confounders.copy()
    X_w_treatment_rem["treatment"] = treatment
    Q_model.fit(X_w_treatment_rem, outcome)
    g_model.fit(remaining_confounders, treatment)
    
    g_list = []
    Q0_list = []
    Q1_list = []
    t_list = []
    out_list = []
    for day in np.unique(wf2020['daynum']):
        df = wf2020[wf2020['daynum'] == day]
        outcome_t = df['diff']
        treatment_t = df['A']
        confounders_t = df[['daynum'] + weather + city_fixed + time_fixed]
        remaining_confounders_t = confounders_t.drop(columns=covs)

        if df['A'].sum() == 0 or num_cities[day] < 2:
            continue

        X1 = remaining_confounders_t.copy()
        X0 = remaining_confounders_t.copy()
        X1["treatment"] = 1
        X0["treatment"] = 0

        Q0 = Q_model.predict(X0)
        Q1 = Q_model.predict(X1)
        g = g_model.predict_proba(remaining_confounders_t)[:,1]
        
        g_list += list(g)
        Q0_list += list(Q0)
        Q1_list += list(Q1)
        t_list += list(treatment_t)
        out_list += list(outcome_t)

#         est, sd = att_aiptw(Q0, Q1, g, treatment_t, outcome_t)
#         res[day] = (est, sd)
    
#     inv_var = np.array([1/v**2 for p,v in res.values()])
#     point = np.array([p for p,v in res.values()])
    
#     tau_hat = (point * inv_var).sum()/inv_var.sum()
#     std_hat = np.sqrt(1/inv_var.sum())
    
    data_and_nuisance_estimates = pd.DataFrame({'g': pd.Series(g_list), 
                                                'Q0': pd.Series(Q0_list), 'Q1': pd.Series(Q1_list), 
                                                'A': pd.Series(t_list), 'Y': pd.Series(out_list)})
    nuisance_estimates[group] = data_and_nuisance_estimates

austen_data_and_nuisance = _convert_to_austen_format(data_and_nuisance_estimates)
austen_data_and_nuisance.to_csv(data_and_nuisance_path, index=False)

pathlib.Path(covariate_dir_path).mkdir(exist_ok=True)
for group, nuisance_estimate in nuisance_estimates.items():
  austen_nuisance_estimate = _convert_to_austen_format(nuisance_estimate)
  austen_nuisance_estimate.to_csv(os.path.join(covariate_dir_path,group+".csv"), index=False)

In [ ]:
austen_data_and_nuisance

In [ ]:
target_bias = 5.00 # note: bias is specified as an absolute number
ap = AustenPlot(data_and_nuisance_path, covariate_dir_path)
p, plot_coords, variable_coords = ap.fit(bias=target_bias)
p